In [1]:
## 导入文件
with open("techFilt_left.vcf","r") as f:
    line_list=f.readlines()
    num=0
    ref=[]
    alt=[]
    chrom=[]
    pos=[]
    genotype=[]
    for i in range(len(line_list)):
        tmp=line_list[i].split()
        chrom.append(tmp[0])
        pos.append(tmp[1])
        ref.append((tmp[3]).split(","))
        alt.append((tmp[4]).split(","))
        ## 取出所有个体的基因型，填入一个二维列表中
        genotype.append([])
        for j in range(9,61):
            tmp2=tmp[j].split(":")
            genotype[num].append(tmp2[0])
        num+=1

### 一、孟德尔分离定律检验

In [2]:
## 标记二等位的snp位点(在这一步去掉indel)
count=0
allele=[]
for i in range(len(alt)):
    if len(alt[i])==1 and len(alt[i][0])==1 and len(ref[i][0])==1:
        count+=1
        allele.append(0)
    else:
        allele.append(99)
print(count/len(alt))

0.8774116076197258


In [3]:
#标记母本基因型是否为杂合子
mtl_heter=[]
for i in range(len(chrom)):
    if genotype[i][0]=='0/1' or genotype[i][0]=='0|1':
        mtl_heter.append(0)
    else:
        mtl_heter.append(99)

#计数子代各种基因型的数量
off_exi=[]
off_zero_zero=[]
off_zero_one=[]
off_one_one=[]
for i in range(len(chrom)):
    null=0
    zero_zero=0
    zero_one=0
    one_one=0
    for j in range(1,52):
        if genotype[i][j]=='./.' or genotype[i][j]=='.|.':
            null+=1
        elif genotype[i][j]=='0/0' or genotype[i][j]=='0|0':
            zero_zero+=1
        elif genotype[i][j]=='0/1' or genotype[i][j]=='0|1':
            zero_one+=1
        elif genotype[i][j]=='1/1' or genotype[i][j]=='1|1':
            one_one+=1    
    off_exi.append(51-null)
    off_zero_zero.append(zero_zero)
    off_zero_one.append(zero_one)
    off_one_one.append(one_one)

In [4]:
# 标记符合孟德尔分离定律的位点，标为0，其余为99.
count=0
seg_test=[]
for i in range(len(chrom)):
    if allele[i]==0 and \
    mtl_heter[i]==0 and \
    (((not off_zero_zero[i]==0) and off_one_one[i]==0) or \
     ((not off_one_one[i]==0) and off_zero_zero[i]==0)):
    ### 0/0 和 1/1必须有其一是0，不然有可能是自交的情况
            
        if off_exi[i]==51:
            if off_zero_one[i]>=19 and off_zero_one[i]<=32:
                count+=1
                seg_test.append(0)
            else:
                seg_test.append(99)
        elif off_exi[i]==50:
            if off_zero_one[i]>=19 and off_zero_one[i]<=31:
                count+=1
                seg_test.append(0)
            else:
                seg_test.append(99)
        elif off_exi[i]==49:
            if off_zero_one[i]>=18 and off_zero_one[i]<=31:
                count+=1
                seg_test.append(0)
            else:
                seg_test.append(99)
        elif off_exi[i]==48:
            if off_zero_one[i]>=18 and off_zero_one[i]<=30:
                count+=1
                seg_test.append(0)
            else:
                seg_test.append(99)
        else:
            seg_test.append(99)
    else:
        seg_test.append(99)
print(count,count/len(chrom),count/len(alt))

791257 0.14086825707673134 0.14086825707673134


In [5]:
for i in range(1564,1565):
    print(chrom[i],pos[i],mtl_heter[i],off_exi[i],off_zero_zero[i],off_zero_one[i],off_one_one[i],seg_test[i])

scaffold1 99979 99 51 27 24 0 99


In [6]:
# 硬删除(all)
with open("techFilt_left.vcf","r") as f:
    line_list=f.readlines()
with open("bioFilt_left.vcf","w") as g:
    g.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tmaternal1"\
            +"\toff05\toff06\toff07\toff_L1_80\toff_L1_81\toff_L1_83\toff_L2_85"\
            +"\toff_L2_86\toff_L2_89\toff_L3_90\toff_L3_92\toff_L3_93\toff_L4_94"\
            +"\toff_L4_96\toff_L4_97\toff_L5_100\toff_L5_101\toff_L5_98\toff_L6_102"\
            +"\toff_L6_106\toff_L6_108\toff_L7_110\toff_L7_111\toff_L7_112\toff_L8_103"\
            +"\toff_L8_113\toff_L8_95\txL1_58\txL1_59\txL1_60\txL2_61\txL2_62\txL2_63"\
            +"\txL3_66\txL3_67\txL3_68\txL4_70\txL4_71\txL4_72\txL5_73\txL5_75\txL5_76"\
            +"\txL6_77\txL6_78\txL6_79\txL7_80\txL7_81\txL7_83\txL8_84\txL8_85\txL8_86\n")
    for i in range(len(line_list)):
            if seg_test[i]==0:
                g.write(line_list[i])
    g.close()

In [9]:
# 取前50kb
with open("techFilt_left.vcf","r") as f:
    line_list=f.readlines()[746:1564] #50kb:745; 100kb:1564
with open("bioFilt_left_50to100kb_v2.vcf","w") as g:
    g.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tmaternal1"\
            +"\toff05\toff06\toff07\toff_L1_80\toff_L1_81\toff_L1_83\toff_L2_85"\
            +"\toff_L2_86\toff_L2_89\toff_L3_90\toff_L3_92\toff_L3_93\toff_L4_94"\
            +"\toff_L4_96\toff_L4_97\toff_L5_100\toff_L5_101\toff_L5_98\toff_L6_102"\
            +"\toff_L6_106\toff_L6_108\toff_L7_110\toff_L7_111\toff_L7_112\toff_L8_103"\
            +"\toff_L8_113\toff_L8_95\txL1_58\txL1_59\txL1_60\txL2_61\txL2_62\txL2_63"\
            +"\txL3_66\txL3_67\txL3_68\txL4_70\txL4_71\txL4_72\txL5_73\txL5_75\txL5_76"\
            +"\txL6_77\txL6_78\txL6_79\txL7_80\txL7_81\txL7_83\txL8_84\txL8_85\txL8_86\n")
    for i in range(len(line_list)):
            if seg_test[i]==0:
                g.write(line_list[i])
    g.close()

In [8]:
##### 画密度图
##取50个最多位点的scaffold，约占全部位点的50%。
##转换为pandas数据框，进行分scaffold计数，并从多到少排序。
from pandas.core.frame import DataFrame
import numpy as np
chrom_left1_all=[]
pos_left1_all=[]
for i in range(len(alt)):
    if seg_test[i]==0:
        chrom_left1_all.append(chrom[i])
        pos_left1_all.append(int(pos[i]))
kept1_dict={"chrom":chrom_left1_all,"pos":pos_left1_all}
kept1_df=DataFrame(kept1_dict)              
kept1_count=kept1_df.groupby(["chrom"]).count()
kept1_count_sort=kept1_count.reset_index().sort_values('pos',ascending = False).set_index('chrom').reset_index()
top50_list=kept1_count_sort['chrom'].tolist()[:50]

##取得top50scaffold的所有位点，写入文件。
chrom_left1_top50=[]
pos_left1_top50=[]
for i in range(0,len(chrom_left1_all)-1):
    if chrom_left1_all[i] in top50_list:
        chrom_left1_top50.append(chrom_left1_all[i])
        pos_left1_top50.append(pos_left1_all[i])

np_chrom_kept=np.array(chrom_left1_top50)
np_pos_kept=np.array(pos_left1_top50)
index=np.arange(1,len(np_pos_kept)+1)
l=np.vstack((index,np_chrom_kept,np_pos_kept)).T
np.savetxt("bioFilt_left_top50_addIndex.txt",l,fmt='%s',delimiter='\t')

### 二、根据符合孟德尔分离定律的二等位位点预测副本基因型
（包括ABxAB, AAxAB, ABxAA, AAxBB的所有二等位位点）